In [1]:
import hashlib
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

review_id = 0   # 21489

def make_user_id(profile_url):
    return hashlib.md5(profile_url.encode()).hexdigest()

def crawl_data(url, res_id):
    global review_id
    browser = webdriver.Chrome()
    browser.get(url)

    results = []

    # Chờ phần review hiển thị
    try:
        WebDriverWait(browser, 15).until(
            EC.presence_of_element_located((By.CLASS_NAME, "review-item"))
        )
    except:
        print(f"⛔ Không tìm thấy review nào cho {url}")
        browser.quit()
        return results

    # Nhấn "Xem thêm bình luận"
    while True:
        try:
            load_more = WebDriverWait(browser, 5).until(
                EC.element_to_be_clickable(
                    (By.XPATH, '//a[@class="fd-btn-more"]/label[contains(text(),"Xem thêm bình luận")]')
                )
            )
            browser.execute_script("arguments[0].click();", load_more)
        except Exception:
            break

    comments = browser.find_elements(By.CLASS_NAME, "review-item")

    for c in comments:
        try:
            user_elem = c.find_element(By.CSS_SELECTOR, ".ru-username")

            username = user_elem.text.strip()
            profile_url = user_elem.get_attribute("href")

            # nếu là relative url
            if profile_url and profile_url.startswith("/"):
                profile_url = "https://www.foody.vn" + profile_url

            user_id = make_user_id(profile_url)

            review_text = c.find_element(By.CSS_SELECTOR, ".rd-des span").text.strip()
            score_text = c.find_element(By.CSS_SELECTOR, ".review-points span").text.strip()
            timestamp = c.find_element(By.CSS_SELECTOR, ".ru-stats span").text.strip()

        except Exception:
            username = "anonymous"
            profile_url = None
            user_id = None
            review_text = "Đã bình luận ẩn danh"
            score_text = "0.0"
            timestamp = None

        review = {
            "review_id": review_id,
            "user_id": user_id,          # ✅ dùng cho CF
            "username": username,        # chỉ để hiển thị
            "profile_url": profile_url,  # debug / trace
            "review_text": review_text,
            "rating": float(score_text),
            "timestamp": timestamp,
            "res_id": res_id,
        }

        review_id += 1
        results.append(review)

    browser.quit()
    return results


In [2]:
from urls import urls

results = []
for idx, url in enumerate(urls, start=800):
    data = crawl_data(url, idx)
    for comment in data:
        results.append(comment)

# Xuất JSON
import json
with open("restaurants_reviews_temp.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)


NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location
